In [1]:
! pip install -q langchain_community
! pip install -q langchain_text_splitter
! pip install -q langchain_openai
! pip install -q faiss-cpu

ERROR: Could not find a version that satisfies the requirement langchain_text_splitter (from versions: none)
ERROR: No matching distribution found for langchain_text_splitter


In [2]:
import os

current_path = os.getcwd()
print("Current path:", current_path)

# colab의 보안 비밀에 OPENAI_API_KEY를 등록해서 환경변수로 설정
#from google.colab import userdata
#
#os.environ['OPENAI_API_KEY'] = userdata.get('OPENAI_API_KEY')

Current path: /Users/khuh/AI-Training/LLM-RAG


In [3]:
import os
import dotenv
from dotenv import load_dotenv

load_dotenv()

# OpenAI API 키 로드
api_key = os.getenv("OPENAI_API_KEY")

In [4]:
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_community.document_loaders.csv_loader import CSVLoader
from langchain_community.vectorstores import FAISS
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough
from langchain_openai import ChatOpenAI, OpenAIEmbeddings

In [ ]:
## from google.colab import drive
## drive.mount('/content/drive')

In [5]:
loader = CSVLoader(file_path="recipies_add_dif-241019.csv", encoding='utf-8')
data = loader.load()
print(f"Loaded {len(data)} documents from the csv.")

Loaded 5612 documents from the csv.


In [6]:
#위에서 불러온 csv 데이터 청크
#RecursiveCharacterTextSplitter를 사용
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=100)

splits = text_splitter.split_documents(data)
len(splits)

5977

In [7]:
from langchain.embeddings import OpenAIEmbeddings
from langchain.vectorstores import FAISS

# 원하는 임베딩 모델 지정
# test-embedding-ada-002, text-embedding-3-small, text-embedding-3-large 사용 가능
embedding_model = OpenAIEmbeddings(model="text-embedding-3-small")
#
# 문서와 함께 FAISS 벡터 스토어 생성
vectorstore = FAISS.from_documents(documents=splits, embedding=embedding_model)
# 소요시간 # 2m 27.3s

/var/folders/py/mwpg_rm904g6ld02xtcmncwh0000gn/T/ipykernel_46416/1859621569.py:6: LangChainDeprecationWarning: The class `OpenAIEmbeddings` was deprecated in LangChain 0.0.9 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-openai package and should be used instead. To use it run `pip install -U :class:`~langchain-openai` and import as `from :class:`~langchain_openai import OpenAIEmbeddings``.
  embedding_model = OpenAIEmbeddings(model="text-embedding-3-small")


In [8]:
### 벡터스토리에서 검색하는 retriever를 만들고 벡터스토어를 로컬에 저장합니다.
### 이미 저장한 경우 이 단계는 건너뜀
## text-embedding-ada-002 모델로 생성한 벡터스토어 저장
vectorstore.save_local('db/faiss241019-3-small')
# 리트리버 생성
retriever = vectorstore.as_retriever()

In [9]:
#from langchain.vectorstores import FAISS
#from langchain.embeddings.openai import OpenAIEmbeddings

# 저장된 벡터스토어 로드
#vectorstore = FAISS.load_local('db/faiss241019-test1', OpenAIEmbeddings(), allow_dangerous_deserialization=True)

# 로드된 벡터스토어에서 retriever 생성
#retriever = vectorstore.as_retriever()

# 검색 예시
query = "토마토"
results = retriever.get_relevant_documents(query)

# 검색 결과 출력
for result in results:
    print(result.page_content)


/var/folders/py/mwpg_rm904g6ld02xtcmncwh0000gn/T/ipykernel_46416/4099274716.py:12: LangChainDeprecationWarning: The method `BaseRetriever.get_relevant_documents` was deprecated in langchain-core 0.1.46 and will be removed in 1.0. Use :meth:`~invoke` instead.
  results = retriever.get_relevant_documents(query)


title: 토마토잼   토마토러버라면 도전
servings: 6 servings
cooking_time: PT30H-1710M
ingredients: 토마토 5개,설탕 200g,소금 1/4숟가락,레몬즙 1/2숟가락
cooking_orders: ['1. 토마토는 열십자로 칼집을 내어 끓는물에 데친다.', '2. 껍질과 꼭지를 제거한다.', '3. 믹서에 간다.', '4. 냄비에 간 토마토, 설탕을 붓고 60분간 끓인다.', '5. 걸쭉한 농도가 되면 소금, 레몬즙을 넣고 10분간 끓인다.', '6. 소독해둔 용기에 담아 완성한다.']
url: https://www.10000recipe.com//recipe/6897583
reason: 조리 시간이 길며, 재료와 조리법이 상당히 많습니다. 또한, 여러 단계의 복잡한 조리 과정이 포함되어 있어 '상'난이도로 판단되었습니다.
difficulties: 상
﻿title: 그릭요거트와 토마토의 만남  그릭토마토샐러드
servings: 1 servings
cooking_time: PT10M
ingredients: 토마토 1개,그릭요거트 1/2종이컵,호두분태 1숟가락,꿀 1숟가락,샐러드채소 1줌,발사믹글레이즈 약간
cooking_orders: ['1. 토마토는 윗 부분을 자르고 속을 파낸다.', '2. 볼에 그릭요거트, 호두분태, 꿀을 넣고 섞는다.', '3. 토마토에 섞은 요거트를 채운다.', '4. 그릇에 샐러드 채소, 그릭토마토를 담고 발사믹글레이즈를 뿌려 완성한다.']
url: https://www.10000recipe.com//recipe/6968519
reason: 조리 시간은 10분으로 적당합니다. 재료의 종류는 7가지이며, 조리 과정은 4단계로 간단합니다. 그러나, 재료들의 조합과 순서에 따라 맛이 달라질 수 있어 어느 정도의 조리 기술이 필요합니다.
difficulties: 중
﻿title: 토마토파르시   홈파티에 딱이네
servings: 2 servings
cooking_time: PT30M
ingredi

In [16]:
from langchain_core.prompts import PromptTemplate

prompt = PromptTemplate.from_template(
    """당신은 질문-답변(Question-Answering)을 수행하는 친절한 AI 어시스턴트입니다. 
    당신의 임무는 주어진 문맥(context) 에서 주어진 질문(question) 에 답하는 것입니다.
    검색된 다음 문맥(context) 을 사용하여 질문(question) 에 답하세요. 
    요리를 추천할 때는 3개의 가장 유사한 요리를 요리 제목과 함께 추천해주세요.
    요리 방법을 답할때는 재료를 목록으로 요리 순서를 목록으로 출력하고 출처도 함께 표시해줘.
    만약, 주어진 문맥(context) 에서 답을 찾을 수 없다면, 답을 모른다면 `주어진 정보에서 질문에 대한 정보를 찾을 수 없습니다` 라고 답하세요.
    항상 한국어로 답변해주세요.

#Question:
{question}

#Context:
{context}

#Answer:"""
)

In [35]:
# !pip install huggingface_hub

In [36]:
# os.environ['HUGGINGFACEHUB_API_TOKEN'] = userdata.get('HF_TOKEN')

In [37]:
#from langchain import LLMChain
#from langchain.prompts import PromptTemplate
#from langchain.llms import HuggingFaceHub
#
# HuggingFace Repository ID
#repo_id = 'Qwen/Qwen2.5-3B-Instruct'
#
## HuggingFaceHub 객체 생성
#llm = HuggingFaceHub(
#    repo_id=repo_id,
#    model_kwargs={"temperature": 0.1, }
#)

# 체인을 생성합니다.
#rag_chain = (
#    {"context": retriever, "question": RunnablePassthrough()}
#    | prompt
#    | llm
#    | StrOutputParser()
#)

In [38]:
# huggingface에서 모델을 가져와서 답변을 생성하는데에 너무 많은 시간이 걸려서 중단 시킴
#question = "30분 이내에 만들 수 있는 요리 추천해줘"
#answer = rag_chain.invoke(question)
#print(answer)

In [17]:
# ChatGPT model 사용
llm = ChatOpenAI(model_name="gpt-4o-mini", temperature=0)

# 체인을 생성합니다.
rag_chain = (
    {"context": retriever, "question": RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
)

In [18]:
# gpt-4o-mini 이용할 경우 수초내에 답변 생성완료
question = "30분 이내에 만들 수 있는 요리 추천해줘"
answer = rag_chain.invoke(question)
print(answer)

30분 이내에 만들 수 있는 요리로 다음의 세 가지를 추천합니다:

1. [요리 1](https://www.10000recipe.com//recipe/6834113)
2. [요리 2](https://www.10000recipe.com//recipe/6949576)
3. [요리 3](https://www.10000recipe.com//recipe/7020135)

각 요리는 조리 시간이 30분으로 상대적으로 짧고, 다양한 재료와 조리 방법이 포함되어 있습니다. 요리의 난이도는 중급 정도로, 초보자에게는 약간의 도전이 될 수 있습니다.


In [19]:
question = "토마토를 이용한 간단한 요리 추천해줘"
answer = rag_chain.invoke(question)
print(answer)

토마토를 이용한 간단한 요리로 다음 세 가지를 추천합니다:

1. **간단한 메밀국수육수로 만든 토마토소바**
   - **재료**: 메밀면 150g, 쪽파 1줄, 김가루 약간, 쯔유 1숟가락, 다시마육수 2종이컵, 방울토마토 150g, 다시마육수 1종이컵, 간장 1숟가락, 설탕 1숟가락, 레몬즙 약간
   - **조리 방법**:
     1. 방울토마토에 십자로 칼집을 낸 후 끓는 물에 데쳐 찬물에 헹군다.
     2. 토마토 껍질을 벗겨 냄비에 넣고 다시마육수, 간장, 설탕, 레몬즙을 넣고 3분간 끓인다.
     3. 토마토조림은 한 김 식힌 후 냉장고에 보관하여 차갑게 둔다.
     4. 쪽파는 송송 썬다.
     5. 쯔유와 다시마육수는 섞어 차갑게 준비한다.
     6. 끓는 물에 메밀면을 삶아 얼음물에 헹군 뒤 물기를 제거한다.
     7. 그릇에 면을 담고 토마토조림, 쪽파, 김가루를 올린 후 육수를 부어 완성한다.
   - [레시피 출처](https://www.10000recipe.com//recipe/6960987)

2. **간단한 안주 토마토카프레제**
   - **재료**: 토마토 2개, 생모짜렐라치즈 2개, 소금 약간, 후추 약간, 시판발사믹소스 1/2종이컵, 다진양파 1숟가락, 어린잎채소 약간
   - **조리 방법**:
     1. 토마토, 생 모짜렐라 치즈는 슬라이스 하여 준비한다.
     2. 토마토와 생 모짜렐라 치즈에 소금, 후추로 밑간을 한다.
     3. 볼에 소스 재료를 넣어 섞어 소스를 만든다.
     4. 그릇에 토마토, 생 모짜렐라 치즈를 번갈아 가며 놓는다.
     5. 소스를 뿌리고 어린잎 채소를 올려 완성한다.
   - [레시피 출처](https://www.10000recipe.com//recipe/6890875)

3. **TV요리 토마토치즈제육볶음**
   - **재료**: 돼지고기 앞다리살 200g, 양파 1/4개, 토마토 1개, 양송이버섯 2개, 청양고추 1개, 토마토 소스 1종이컵,

In [20]:
question = "두부를 이용한 요리를 하나 추천하고 조리 방법을 알려주고 출처도 표시해줘"
answer = rag_chain.invoke(question)
print(answer)

추천하는 요리는 **두부조림**입니다. 아래는 두부조림의 조리 방법입니다.

### 재료
- 두부 290g
- 진간장 7스푼
- 송송썰은 파 3스푼
- 다진 마늘 1/3스푼
- 참기름 1/2스푼
- 통깨 1/3스푼
- 고춧가루 1/2스푼
- 설탕 1/2스푼
- 물 50ml
- 식용유 1스푼

### 조리 순서
1. 두부를 도톰한 두께로 썰어 주세요.
2. 식용유 1스푼을 두른 팬에 노릇하게 부쳐서 준비합니다.
3. 분량의 진간장, 파, 다진 마늘, 참기름, 통깨, 고춧가루, 설탕을 넣고 양념장을 만들어 줍니다.
4. 냄비에 사진처럼 노릇하게 부쳐낸 두부를 한 켜 깔고, 양념장을 올려줍니다.
5. 물 50ml를 넣고, 강불에서 국물이 자작해질 때까지 졸여줍니다.
6. 끓이는 중간중간에 수저로 국물을 떠서 두부에 얹어주면 양념이 좀 더 잘 베인답니다. 두부조림 완성.

출처: [10,000 Recipe](https://www.10000recipe.com//recipe/4102401)


In [21]:
question = "짜장면 만드는 방법 알려줘"
answer = rag_chain.invoke(question)
print(answer)

짜장면 만드는 방법은 다음과 같습니다.

### 재료
- 비비고 한섬만두 4개
- 칼국수 면 2인분
- 춘장 3숟가락
- 찹쌀가루 1숟가락
- 양파 1개
- 대파 1대
- 생강 1톨
- 설탕 2숟가락
- 물 2종이컵

### 요리 순서
1. 한섬 만두는 실온에 꺼내두어 자연 해동한다.
2. 양파, 대파는 한입 크기로 큼직하게 썰어 준비하고 생강은 편으로 썰어 준비한다.
3. 해동시킨 만두는 반을 갈라 속을 파내어 준비한다.
4. 팬에 기름을 두르고 양파, 대파, 생강 순서로 볶는다.
5. 양파가 투명해지면 준비한 만두속을 넣어 볶는다.
6. 볶아진 재료에 춘장, 설탕을 넣어 볶는다.
7. 재료들이 잘 섞이면 물을 넣어 한소끔 끓인 후 찹쌀가루를 넣어 농도를 맞춘다.
8. 면은 끓는 물에 삶아 내어 준비한다.
9. 그릇에 면과 짜장 소스를 담아 짜장면을 완성한다.

출처: [10,000 Recipe](https://www.10000recipe.com//recipe/6881207)

추가로 추천할 수 있는 유사한 요리는 다음과 같습니다:
1. **쫄면** - 쫄면과 다양한 채소를 곁들인 요리
2. **만두소짜장라면** - 만두소를 활용한 짜장라면
3. **냉라면** - 시원한 냉국과 함께 즐기는 라면
